This notebook scrapes Wikipedia for Toronto postal data and assigns geographic coordinates using geocoder.arcgis. We then retrieve Foursquare data on venues in Toronto proper, and use the K-Means clustering algorithm to cluster neighborhoods based on the results.

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install geocoder
import geocoder
print('Libraries imported.')

Libraries imported.


The following code scapes Wikipedia for Toronto postal code data and casts it as a Pandas dataframe:


In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#results=requests.get(url).json()
results=pd.read_html(url)
df=results[0]
print("The shape of the dataframe is: ", df.shape)
df.head()

The shape of the dataframe is:  (180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The following code cleans the dataframe:

In [3]:
#drop rows in which Borough is "Not Assigned"
indexes = df[df['Borough'] == 'Not assigned' ].index
df.drop(indexes , inplace=True)
df.reset_index(drop=True,inplace=True)
print("unique values of boroughs: ", df.Borough.unique())

#rename Postal Code column
df.rename(columns={'Postal Code': 'Postal_Code'}, inplace=True)

#ensure uniquness of postal codes
max=df['Postal_Code'].value_counts(dropna=False).max()
assert max==1

#replace "/" with commas in Neighborhood column
df['Neighborhood']=df['Neighborhood'].str.replace(" /", ",")

#Ensure that Neighboorhood is always assigned
print(df.Neighborhood.isna().sum())
df.head()



unique values of boroughs:  ['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']
0


,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Determine the shape of the resulting dataframe:

In [4]:
df.shape

(103, 3)

Add columns for latitude and longitude to the dataframe:

In [5]:
#define function to retrieve coordinates
def get_latlon(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

cord=get_latlon('M4A') # test function

#add empty columns and iterate over rows to fill coordinates
df['Latitude']=np.nan
df['Longitude']=np.nan
for row_index,row in df.iterrows():
    cord=get_latlon(row['Postal_Code'])
    print(row['Postal_Code'],cord)
    df.loc[row_index,'Latitude']=cord[0]
    df.loc[row_index,'Longitude']=cord[1]
df.tail()


M3A [43.75293455500008, -79.33564142299997]
M4A [43.72810248500008, -79.31188987099995]
M5A [43.65096410900003, -79.35304116399999]
M6A [43.723265465000054, -79.45121077799996]
M7A [43.66179000000005, -79.38938999999993]
M9A [43.66748067300006, -79.52895286499995]
M1B [43.80862623100006, -79.18991284599997]
M3B [43.74890000000005, -79.35721999999998]
M4B [43.70719267700008, -79.31152927299996]
M5B [43.65749059800004, -79.37752923699998]
M6B [43.70727872700007, -79.44750009299997]
M9B [43.65002250300006, -79.55408903099999]
M1C [43.78577865700004, -79.15736763799998]
M3C [43.72214339800007, -79.35202341799999]
M4C [43.68974004200004, -79.30850701899999]
M5C [43.65173364700007, -79.37555358799995]
M6C [43.69172991700003, -79.43001279899994]
M9C [43.637813150000056, -79.57648363299995]
M1E [43.76580607300008, -79.18528434099994]
M4E [43.67814827600006, -79.29534930999995]
M5E [43.645195888000046, -79.37385548899994]
M6E [43.68911756600005, -79.45065043699998]
M1G [43.77154467100007, -79.2

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,Business reply mail Processing Centre,43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000


Check coordinates for accuracy:

In [6]:
cord=get_latlon('M2H')
cord

[43.802556149000054, -79.35656576499997]

Let's visualize the data and generate a map of neighborhoods in Toronto, Ontario:

In [7]:
# create map of Toronto neighborhoods using latitude and longitude values
address="Toronto, Ontario"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let's restrict our dataset to only boroughs which contain the word "Toronto." We will then retrieve data on the venues located in each neighborhood using the Foursquare API. Finally, we will cluster the neighborhoods based on those venues:

In [8]:
#df_downtown=df[df['Borough']=="Downtown Toronto"].reset_index(drop=True)
df_toronto=df[df['Borough'].str.contains('Toronto')==True].reset_index(drop=True)
#print("Shape of Downtown Toronto dataset: ", df_downtown.shape)
print("Shape of Toronto dataset: ", df_toronto.shape)
df_toronto['Borough'].value_counts()


Shape of Toronto dataset:  (39, 5)


Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

Input user credentials to retrieve Foursquare data:

In [9]:
CLIENT_ID = 'SDCNWVQ20RISRQVXCIWQP1NRT0XWZTHXSEI4NKCAOH0D1LGE' # your Foursquare ID
CLIENT_SECRET = 'FCRTWP1WTSVFPLC0NTJV32RJTW1GTBBNC0VJSWR5EVMEV1MY' # your Foursquare Secret
VERSION = '20200504'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SDCNWVQ20RISRQVXCIWQP1NRT0XWZTHXSEI4NKCAOH0D1LGE
CLIENT_SECRET:FCRTWP1WTSVFPLC0NTJV32RJTW1GTBBNC0VJSWR5EVMEV1MY


Define a function to retrieve Foursquare data on venues in Downtown Toronto:

In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Retrieve Foursquare data of venues in Downtown Toronto:

In [11]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [12]:
print(toronto_venues.shape)
toronto_venues.tail()


(1603, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1598,Business reply mail Processing Centre,43.6487,-79.38545,Rexall Pharma Plus,43.650693,-79.386847,Pharmacy
1599,Business reply mail Processing Centre,43.6487,-79.38545,Ematei 絵馬亭,43.651156,-79.388990,Japanese Restaurant
1600,Business reply mail Processing Centre,43.6487,-79.38545,Meli Baklava & Chocolate Bar,43.650140,-79.390377,Chocolate Shop
1601,Business reply mail Processing Centre,43.6487,-79.38545,Red Eye Espresso,43.651150,-79.390146,Café
1602,Business reply mail Processing Centre,43.6487,-79.38545,Condom Shack,43.650542,-79.388138,Hobby Shop


Let's check how many venues were returned for each neighborhood:

In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,66,66,66,66,66,66
"Brockton, Parkdale Village, Exhibition Place",44,44,44,44,44,44
Business reply mail Processing Centre,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",66,66,66,66,66,66
Central Bay Street,52,52,52,52,52,52
Christie,12,12,12,12,12,12
Church and Wellesley,83,83,83,83,83,83
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,29,29,29,29,29,29


How many unique categories can be curated from all the returned venues?

In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 226 uniques categories.


One-hot encoding:

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
toronto_onehot.tail()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
1598,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Business reply mail Processing Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1599,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,

Let's group rows by Neighborhood and take the mean of the frequency of each category:

In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.015152,0.000000,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.015152,0.030303,0.000000,0.000000,0.00,0.015152,0.000000,0.030303,0.000000,0.00000,0.015152,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.015152,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.00,0.015152,0.045455,0.090909,0.000000,0.00000,0.00000,0.000000,0.00,0.00,0.015152,0.000000,0.015152,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.015152,0.015152,0.000000,0.015152,0.00000,0.000000,0.000000,0.000000,0.015152,0.000000,0.00,0.000000,0.000000,0.015152,0.015152,0.000000,0.015152,0.000000,0.000000,0.000000,0.00,0.015152,0.015152,0.015152,0.00

Let's print each neighborhood along with the top 5 most common venues:

In [17]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Beer Bar  0.03
3        Bakery  0.03
4          Café  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.07
2               Gift Shop  0.05
3  Thrift / Vintage Store  0.05
4                   Diner  0.05


----Business reply mail Processing Centre----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2  Japanese Restaurant  0.04
3                 Café  0.04
4           Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
               venue  freq
0               Café  0.06
1        Coffee Shop  0.06
2               Park  0.05
3         Restaurant  0.05
4  French Restaurant  0.05


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.15
1  Japanese R

Let's write a function to sort the venues in descending order:

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create a new dataframe and display the top 10 venues for each neighborhood:

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Breakfast Spot,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Hotel,Restaurant,Café
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Diner,Pizza Place,Gift Shop,Thrift / Vintage Store,Pet Store,Indian Restaurant,Italian Restaurant,Boutique
2,Business reply mail Processing Centre,Coffee Shop,Hotel,Restaurant,Café,Japanese Restaurant,Asian Restaurant,Tea Room,Salon / Barbershop,Sandwich Place,Bookstore
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Park,Restaurant,French Restaurant,Gym / Fitness Center,Lounge,Italian Restaurant,Speakeasy,Bar
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Plaza,Restaurant,Café,Bubble Tea Shop,Sandwich Place,Clothing Store,Middle Eastern Restaurant,Italian Restaurant


We will now run the k-means clustering algorithm to cluster the neighborhoods into 5 clusters:

In [20]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster, as well as the top  10 venues for each neighborhood:

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#clean the dataframe
toronto_merged['Cluster Labels'].value_counts(dropna=False)
print(toronto_merged.shape)
toronto_merged.dropna(axis=0, inplace=True)
print(toronto_merged.shape)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int64')
toronto_merged.head()

(103, 16)
(39, 16)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,1,Pub,Café,Athletics & Sports,Bakery,Tech Startup,Mediterranean Restaurant,Mexican Restaurant,Food Truck,French Restaurant,Chocolate Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,1,Coffee Shop,Sushi Restaurant,Café,Hobby Shop,Discount Store,Restaurant,Pharmacy,Park,Middle Eastern Restaurant,Juice Bar
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,1,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Restaurant,Hotel,Italian Restaurant,Café,Bar,Cosmetics Shop
15,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,1,Coffee Shop,Café,Seafood Restaurant,Cocktail Bar,American Restaurant,Cosmetics Shop,Gastropub,Restaurant,Clothing Store,Theater
19,M4E,East Toronto,The Beaches,43.678148,-79.295349,1,Health Food Store,Trail,Pub,Church,Discount Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Now let's visualize the resulting clusters:

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can now examine the resulting clusters:

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Park,Grocery Store,Bus Line,Business Service,Discount Store,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
68,Central Toronto,0,Gym / Fitness Center,Park,Women's Store,Distribution Center,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
73,Central Toronto,0,Playground,Gym Pool,Park,Garden,Distribution Center,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
91,Downtown Toronto,0,Playground,Grocery Store,Park,Candy Store,Distribution Center,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [24]:
toronto_merged.rename(columns={'Cluster Labels': 'Cluster'}, inplace=True)
toronto_merged['Cluster'].value_counts()

1    32
0     4
4     1
3     1
2     1
Name: Cluster, dtype: int64

In [25]:
cluster0 = toronto_merged[toronto_merged.Cluster == 0]
cluster0['1st Most Common Venue'].value_counts()

Playground              2
Park                    1
Gym / Fitness Center    1
Name: 1st Most Common Venue, dtype: int64

In [26]:
cluster4 = toronto_merged[toronto_merged.Cluster == 4]
cluster4['1st Most Common Venue'].value_counts()

Swim School    1
Name: 1st Most Common Venue, dtype: int64

In [27]:
cluster1 = toronto_merged[toronto_merged.Cluster == 1]
cluster1['1st Most Common Venue'].value_counts()

Coffee Shop            17
Café                    4
Sandwich Place          2
Dessert Shop            1
Government Building     1
Tennis Court            1
Pub                     1
Park                    1
Food & Drink Shop       1
Restaurant              1
Playground              1
Health Food Store       1
Name: 1st Most Common Venue, dtype: int64

In [28]:
cluster2 = toronto_merged[toronto_merged.Cluster == 2]
cluster2['1st Most Common Venue'].value_counts()

Spa    1
Name: 1st Most Common Venue, dtype: int64

In [29]:
cluster3 = toronto_merged[toronto_merged.Cluster == 3]
cluster3['1st Most Common Venue'].value_counts()

Theme Park    1
Name: 1st Most Common Venue, dtype: int64

We can see that the clustering algorithm segments neighborhoods based on venues offering opportunities for physical activity versus venues providing hospitality and food services. I hope you enjoyed this exercise!